In [1]:
# Install environment and agent
!pip install highway-env
# TODO: we use the bleeding edge version because the current stable version does not support the latest gym>=0.21 versions. Revert back to stable at the next SB3 release.
!pip install git+https://github.com/carlosluis/stable-baselines3@75fd27e019aaf4dad612823b4544620df2c47844

  Cloning https://github.com/carlosluis/stable-baselines3 (to revision 75fd27e019aaf4dad612823b4544620df2c47844) to c:\users\caste\appdata\local\temp\pip-req-build-zc7cf0yt
  Resolved https://github.com/carlosluis/stable-baselines3 to commit 75fd27e019aaf4dad612823b4544620df2c47844
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/carlosluis/stable-baselines3 'C:\Users\caste\AppData\Local\Temp\pip-req-build-zc7cf0yt'
  Running command git rev-parse -q --verify 'sha^75fd27e019aaf4dad612823b4544620df2c47844'
  Running command git fetch -q https://github.com/carlosluis/stable-baselines3 75fd27e019aaf4dad612823b4544620df2c47844
  Running command git checkout -q 75fd27e019aaf4dad612823b4544620df2c47844


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Environment
import gym
import highway_env

# Agent
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.evaluation import evaluate_policy # Test models

In [4]:
env = gym.make("racetrack-v0")

Primeiramente, vamos treinar agentes dados parâmetros mais comuns e avaliar os resultados obtidos para cada um.
Posteriormente, para os mesmos algoritmos testados, usaremos o `Optuna` para tunar hiperparâmetros e avaliá-los. Por fim, criaremos agentes de tais algoritmos usando hiperparâmetros tunados no <a href= "https://github.com/DLR-RM/rl-baselines3-zoo">RL Zoo</a> para problemas semelhantes. Os algoritmos usados serão o `PPO` e o `SAC`.

## Avaliando modelos com os parâmetros padrões

In [6]:
# Agents with default parameters
sac_model_default = SAC('MlpPolicy', env)
ppo_model_default = PPO('MlpPolicy', env)

In [10]:
from typing import Any, Dict

def learn_and_evaluate(model: Any, env: gym.Env, n_timesteps: int | float, n_episodes: int, default: bool, algorithms_results: dict) -> Dict[str, dict]:
    """Function destinated to train and evaluate agents"""
    
    if default:
        string_aux = ' default'
    else:
        string_aux = ' tunned'
    model_name = str(model.__class__).split('.')[-1][:3] + string_aux

    model.learn(n_timesteps)
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=n_episodes)
    algorithms_results[model_name] = {}
    algorithms_results[model_name]['mean_reward'] = mean_reward
    algorithms_results[model_name]['std_reward'] = std_reward

    return algorithms_results

algorithms_results = {}

In [8]:
algorithms_results = learn_and_evaluate(sac_model_default, env, 2e4, 10, default=True, algorithms_results=algorithms_results)
algorithms_results = learn_and_evaluate(ppo_model_default, env, 5e4, 10, default=True, algorithms_results=algorithms_results)

algorithms_results

{'SAC default': {'mean_reward': 449.29054162614045,
  'std_reward': 481.6294687877488},
 'PPO default': {'mean_reward': 525.7549042252824,
  'std_reward': 438.0120456032163}}

In [9]:
# Saving trained models
sac_model_default.save(path='models/sacDefault')
ppo_model_default.save(path='models/ppoDefault')

In [10]:
# Load of the agents saved above
sac_default_save = SAC.load('models/sacDefault')
ppo_default_save = PPO.load('models/ppoDefault')

In [11]:
def run_model(env: gym.Env, model: Any) -> None:
    """Function to run a model and render the enviroment through time"""
    
    obs, done = env.reset(), False

    i = 0
    while not(done) and i < 300:
        optAction, _states = model.predict(obs)
        obs, reward, done, info = env.step(optAction)
        env.render()
        i += 1

    env.close()

Agora, vamos usar a função acima para vermos como os agentes estão se comportando no ambiente.

In [16]:
# PPO simulation
run_model(env, ppo_default_save)

In [14]:
# SAC simulation
run_model(env, sac_default_save)

Analisando o obtido, vemos que o `PPO` não se comportou tão bem, pois conseguia manter uma velociade alta, mas a direção do carro parecia meio aleatória. Já o `SAC` tinha uma velocidade menor, mas tinha uma direção mais controlada, mas, mesmo assim, não é capaz de desviar do outro carro quando ambos estão na mesma faixa. Com isso, percebemos que nenhum dos dois é satisfatório para o proposto.

Agora, faremos processo similar para modelos com parâmetros tunados, começando pelo `SAC`.

## Avaliando modelos com otimização de hiperparâmetros
### SAC

Primeiro, faremos um estudo rápido a fim de, apenas, descobrir os parâmetros mais relevantes.

In [6]:
import optuna
from optimizerSAC import *

In [5]:
sac_test_study = optuna.create_study(
    sampler=optuna.samplers.TPESampler(), pruner=None, direction='maximize', storage="sqlite:///sacTest.db",
    study_name="sacTest", load_if_exists=True
)
sac_test_study.optimize(lambda trial: objectiveSAC(trial, 2, 5e3, 5), n_trials=5, catch=(AssertionError, ValueError))

[I 2022-11-05 18:30:42,420] A new study created in RDB with name: sacTest
[I 2022-11-05 18:53:44,051] Trial 0 finished with value: 107.30800413042307 and parameters: {'gamma': 0.95, 'learning_rate': 0.01, 'batch_size': 512, 'buffer_size': 100000, 'train_freq': 256, 'tau': 0.05, 'net_arch': 'medium'}. Best is trial 0 with value: 107.30800413042307.
[I 2022-11-05 19:12:43,037] Trial 1 finished with value: 20.09597761631012 and parameters: {'gamma': 0.9, 'learning_rate': 0.001, 'batch_size': 512, 'buffer_size': 10000, 'train_freq': 32, 'tau': 0.01, 'net_arch': 'small'}. Best is trial 0 with value: 107.30800413042307.
[I 2022-11-05 19:24:49,072] Trial 2 finished with value: 257.2470165563747 and parameters: {'gamma': 0.98, 'learning_rate': 0.01, 'batch_size': 64, 'buffer_size': 10000, 'train_freq': 16, 'tau': 0.001, 'net_arch': 'small'}. Best is trial 2 with value: 257.2470165563747.
[I 2022-11-05 19:44:42,721] Trial 3 finished with value: 15.001020169258117 and parameters: {'gamma': 0.98,

In [6]:
from optuna.importance import get_param_importances
get_param_importances(sac_test_study)

OrderedDict([('learning_rate', 0.20952476458406977),
             ('train_freq', 0.19988291643976006),
             ('batch_size', 0.1932064535823394),
             ('tau', 0.1785067660504574),
             ('gamma', 0.14762868394240056),
             ('net_arch', 0.05310901917563603),
             ('buffer_size', 0.01814139622533673)])

Com isso, vemos que o `learning_rate` foi o mais importante para a otimização, enquanto que `buffer_size` e `net_arch` foram os menos influentes, assim, para a otimização do modelo, vamos desconsiderar estes parâmetros e trabalhar com os seus valores padrões. Só vamos excluí-los somente porque suas importâncias foram extremamente baixas e  pois é necessário excluir alguns parâmetros devido ao elevado tempo de execução da otimização, porque uma quantidade pequena de *trials* não deve ser muito conclusiva. Agora vamos rodar com mais *timesteps* e uma maior quantidade de *trials*, a fim de encontrar os melhores hiperparâmetros dentre os listados, para, posteriormente, criarmos um agente com tais valores.

In [6]:
# SAC Optimization study
sac_study = optuna.create_study(
    sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_startup_trials=5), direction='maximize',
    study_name='sacStudy', storage="sqlite:///sacstudy.db", load_if_exists=True
)

[I 2022-11-06 10:28:19,598] A new study created in RDB with name: sacStudy


In [ ]:
# Optimizing SAC hyperparameters
sac_study.optimize(
    lambda trial: objectiveSAC(trial, n_evaluations=4, n_timesteps=12e3, n_eval_episodes=15), n_trials=8, catch=(AssertionError, ValueError)
)

Devido a um limite de memória para o estudo, conseguiu-se fazer 8 *trials*, apenas, e o resultado foi o abaixo:

In [16]:
# Information about SAC best case
trial = sac_study.best_trial

print("Value: ", trial.value)

print("Params: ")
for key, value in trial.params.items():
    print(f"  {key}: {value}")

Value:  846.5468435429036
Params: 
  batch_size: 64
  gamma: 0.9
  learning_rate: 0.01
  tau: 0.01
  train_freq: 128


Agora, com os hiperparâmetros tunados do `SAC`, vamos criar um agente com tais parâmetros e treiná-los com 20.000 *timesteps*, uma quantidade  maior que a usada durante a otimização e depois ver como ele está se comportando no ambiente.

In [ ]:
opt_sac = SAC(
    policy='MlpPolicy', env=env, gamma=0.9, batch_size=64, learning_rate=0.01, train_freq=128,
    gradient_steps=128, tau=0.01
)
opt_algorithms_results = {}
learn_and_evaluate(
    opt_sac, env=env, n_timesteps=3e4, n_episodes=20, default=False, algorithms_results=opt_algorithms_results
)

In [15]:
opt_algorithms_results

{'SAC tunned': {'mean_reward': 777.3178841821849,
  'std_reward': 647.9604158437256}}

In [14]:
opt_sac.save("models/sacTuned")

In [28]:
# Loading SAC tuned model
opt_sac_save = SAC.load("models/sacTuned")

In [29]:
run_model(env=env, model=opt_sac_save)

Vendo seu comportamento no ambiente, vemos que o agente continua incapaz de desviar do outro carro, quando ambos os carros estão na mesma faixa. Entretanto, percebe-se uma melhora com relação ao primeiro agente criado, principalmente, quanto à velocidade atingida pelo carro. Assim, mesmo com a melhora significativa no retorno esperado médio, o agente continua ineficaz.

### PPO

Primeiro, faremos um estudo rápido a fim de, apenas, descobrir os parâmetros mais relevantes.

In [7]:
from optimizerPPO import *

In [9]:
ppo_test_study = optuna.create_study(
    sampler=optuna.samplers.TPESampler(), pruner=None, direction='maximize', storage="sqlite:///ppoTest.db",
    study_name="ppoTest", load_if_exists=True
)
ppo_test_study.optimize(lambda trial: objectivePPO(trial, 2, 1e4, 5), n_trials=5, catch=(AssertionError, ValueError))

[I 2022-11-05 20:12:07,422] A new study created in RDB with name: ppoTest
[I 2022-11-05 20:28:41,643] Trial 0 finished with value: 94.08841642215847 and parameters: {'batch_size': 32, 'n_steps': 32, 'gamma': 0.98, 'learning_rate': 1.6434419844976428e-05, 'ent_coef': 1.8870841559669228e-08, 'clip_range': 0.3, 'n_epochs': 10, 'gae_lambda': 0.92, 'net_arch': 'medium'}. Best is trial 0 with value: 94.08841642215847.
[I 2022-11-05 20:44:40,774] Trial 1 finished with value: 258.88079236522316 and parameters: {'batch_size': 8, 'n_steps': 8, 'gamma': 0.98, 'learning_rate': 0.008405684599925403, 'ent_coef': 2.915164906645702e-05, 'clip_range': 0.4, 'n_epochs': 20, 'gae_lambda': 0.92, 'net_arch': 'small'}. Best is trial 1 with value: 258.88079236522316.
[I 2022-11-05 20:55:25,382] Trial 2 finished with value: 221.96929724290968 and parameters: {'batch_size': 64, 'n_steps': 128, 'gamma': 0.995, 'learning_rate': 0.044335119736724056, 'ent_coef': 2.885210424618312e-07, 'clip_range': 0.1, 'n_epochs'

In [10]:
get_param_importances(ppo_test_study)

OrderedDict([('batch_size', 0.18791876109031888),
             ('learning_rate', 0.16264030721053566),
             ('n_epochs', 0.14157810998445752),
             ('n_steps', 0.1369247397346501),
             ('ent_coef', 0.12533860051416498),
             ('gamma', 0.08847227753641605),
             ('clip_range', 0.0810378741656116),
             ('gae_lambda', 0.04799848776046512),
             ('net_arch', 0.028090842003380103)])

Vendo o obtido, percebe-se que `batch_size` foi o mais relevante, entretanto, a maioria dos hiperparâmetros apresentaram importância similar. Aqueles menos relevantes foram `net_arch` e `gae_lambda`, assim, pelos motivos explicados na otimização do `SAC`, desconsideraremos eles para achar os melhores valores dos parâmetros. Agora, faremos outro estudo com mais *trials* e *timesteps*.

In [8]:
# PPO Optimization study
ppo_study = optuna.create_study(
    sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_startup_trials=5), direction='maximize',
    study_name='ppoStudy', storage="sqlite:///ppostudy.db", load_if_exists=True
)

[I 2022-11-07 20:09:46,677] Using an existing study with name 'ppoStudy' instead of creating a new one.


In [ ]:
# Optimizing PPO hyperparameters
ppo_study.optimize(lambda trial: objectivePPO(trial, 4, 15e3, 15), n_trials=30, catch=(AssertionError, ValueError))

In [11]:
# Information about PPO best case
trial = ppo_study.best_trial

print("Value: ", trial.value)

print("Params: ")
for key, value in trial.params.items():
    print(f"  {key}: {value}")


Value:  616.6054743170738
Params: 
  batch_size: 128
  clip_range: 0.1
  ent_coef: 0.01
  gamma: 0.95
  learning_rate: 0.0001
  n_epochs: 20
  n_steps: 32


Com os hiperparâmetros tunados, vamos criar o agente do algoritmo `PPO`, treiná-lo com os parâmetros encontrados, e ver o resultado no ambiente.

In [18]:
opt_ppo = PPO(
    policy='MlpPolicy', env=env, seed=5, gamma=0.95, batch_size=128, learning_rate=0.0001, n_epochs=20,
    n_steps=32, ent_coef=0.01, clip_range=0.1
)
opt_algorithms_results ={}
learn_and_evaluate(
    opt_ppo, env=env, n_timesteps=1e5, n_episodes=20, default=False, algorithms_results=opt_algorithms_results
)
opt_ppo.save("models/ppoTuned")

In [23]:
opt_algorithms_results

{'PPO tunned': {'mean_reward': 678.5849429577589,
  'std_reward': 678.0447825283173}}

In [18]:
# Loading PPO tuned model
opt_ppo_save = PPO.load("models/ppoTuned")

In [20]:
run_model(env=env, model=opt_ppo_save)

Acerca do obtido, podemos tirar conclusões semelhantes às feitas para o `SAC`, ou seja, que o agente apresentou melhora no quesito velocidade, entretanto, ainda não consegue desviar do outro carro. Comparado ao agente com parâmetros padrões, vemos que a melhora no controle da direção é significativa. Portanto, o modelo criado não atende ao que se espera dele.

## Agentes com hiperparâmetros do RL Zoo
### SAC
Os hiperparâmetros tunados para o `SAC` de alguns ambientes do *gym*, os quais foram considerados para essa parte, encontram-se <a href="https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/hyperparams/sac.yml">aqui</a>. Os hiperparâmetros que servirão de base para testarmos nesse ambiente, são aqueles designados para a resolução do ambiente `MountainCarContinuous`, devido a uma certa similaridade entre esse ambiente e o `Highway` e por não ter uma quantidade muito alta de *timesteps*.

In [7]:
# Model with MountainCarContinuous tuned hyperparameters
sac_mountain_zoo = SAC(
    policy='MlpPolicy', env=env, learning_rate=3e-4, buffer_size=50000, batch_size=512, 
    ent_coef=0.1, gamma=0.9999, tau=0.01, train_freq=32, gradient_steps=32, learning_starts=0, use_sde=True, 
    policy_kwargs=dict(log_std_init=3.67, net_arch=[64, 64])
)
zoo_algorithms_results = {}
learn_and_evaluate(sac_mountain_zoo, env=env, n_timesteps=5e4, n_episodes=30, default=False, algorithms_results=zoo_algorithms_results)
sac_mountain_zoo.save("models/sacMountainZoo")
zoo_algorithms_results

{'SAC tunned': {'mean_reward': 740.5834671095207,
  'std_reward': 526.2893797126395}}

In [5]:
sac_mountain_zoo_save = SAC.load("models/sacMountainZoo")

In [21]:
run_model(env, sac_mountain_zoo_save)

Esse foi o melhor modelo até agora, o carro consegue andar numa velocidade boa e é o primeiro agente que ao se aproximar do carro da frente tenta trocar de direção. O único problema é que nem sempre essa troca de direção é feita da maneira correta, as vezes ele acaba por dar uma volta e andar no sentido oposto, em outras ele foge muito dos limites da pista, e em outras ele faz a ultrapassagem corretamente.

### PPO
Os hiperparâmetros tunados para o `PPO` de alguns ambientes do *gym*, os quais foram considerados para essa parte, encontram-se <a href="https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/hyperparams/ppo.yml">aqui</a>. Os hiperparâmetros que servirão de base para testarmos nesse ambiente, são aqueles designados para a resolução do ambiente `MountainCarContinuous`, pelos mesmos motivos já apresentados.

In [8]:
ppo_mountain_zoo = PPO(
    policy='MlpPolicy', env=env, learning_rate=7.7e-5, n_steps=8, batch_size=256, n_epochs=10, ent_coef=0.00429, 
    gamma=0.9999, gae_lambda=0.9, max_grad_norm=5, vf_coef=0.19,use_sde=True, clip_range=0.1,
    policy_kwargs=dict(log_std_init=-3.29, ortho_init=False)
)
zoo_algorithms_results = {}
learn_and_evaluate(ppo_mountain_zoo, env=env, n_episodes=30, n_timesteps=2e4, default=False, algorithms_results=zoo_algorithms_results)
ppo_mountain_zoo.save("models/ppoMountainZoo")
zoo_algorithms_results

{'PPO tunned': {'mean_reward': 215.40163235133514,
  'std_reward': 227.03301917839454}}

In [22]:
ppo_mountain_zoo_save = PPO.load("models/ppoMountainZoo")

In [23]:
run_model(env=env, model=ppo_mountain_zoo_save)

Podemos ver que o resultado obtido foi pior do que o agente os com parâmetros padrões.
## Conclusão
Depois de treinarmos e observamos agentes em 3 situações distintas, uma com hiperparâmetros padrões, outra tunando alguns parâmetros, e, por fim, treinando um agente com hiperparâmetros já tunados em um ambiente similar, podemos concluir o seguinte:
<ul>
    <li>Comparando todos os agentes treinados, o melhor foi o SAC com parâmetros tunados do MountainCarContinuous;  </li>
    <li>Apesar de observarmos uma melhora nos modelos com hiperparâmetros tunados localmente, no caso do PPO foi o melhor resultado, fazer tal otimização é muito custosa, por isso tivemos que restringir a quantidade de parâmetros considerados e a quantidade de timesteps. Assim, percebe-se que obter um resultado ótimo, de fato, não é algo tão simples de ser realizado;</li>
</ul>